 **Workspace Setup**

 For this summarization model, you must adjust your runtime space from CPU to GPU. We do this for the express purpose of reducing total runtime for the summarization model [Adjust this explaination]

To switch from CPU to GPU, select:
Runtime > Change runtime type > T4 GPU



**Necessary User Imports and Downloads**

In [1]:
!pip install PyMuPDF -q
!pip install -U datasets transformers ninja -q
!pip install -U sentencepiece -q
!pip install clean-text[gpl] -q
!pip install ads -q


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
  DEPRECATION: emoji is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import time
import multiprocessing
import re
import requests
import os
import concurrent.futures
import fitz

#For summarization
from transformers import pipeline
import torch
from cleantext import clean
from pathlib import Path
import logging
from datasets import load_dataset
from tqdm.auto import tqdm
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

#From Expertise Finder v2.0
import TextAnalysis as TA

#For N-grams
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\izzye\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\izzye\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**Citing this code**

This code is an enhancement or addition to the Expertise finding tool developed by Volz et al. 2023. When citing this particular summarization code, please use Hoare et al. 2024.

Import .csv file produced from Expertise Finder v2.0 code (*Helfenbein et al. 2023)*. This code will only work with results produced by ADS_search.

In [4]:
datf = pd.read_csv('output.csv')
datf.set_index(datf.columns[0], inplace=True, drop=True)

datf

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
Unnamed: 0,,,,,,,,,,,,,,
0,"Browning, Matthew","University of California, Berkeley","Enoch, Melissa L., Hull, A. J., Oka, M., Salem...","2009ApJ...707..103E, 2009AGUFMSM41B1727H, 2009...",['Disk and Envelope Structure in Class 0 Proto...,"2009-12-00, 2009-12-00, 2009-12-00, 2009-12-00...","['infrared: ISM', 'ISM: individual: Serpens', ...","Department of Astronomy, University of Califor...",We present the first results of a program to c...,"['arXiv:0910.2715', '10.1088/0004-637X/707/1/1...","[('field', 1196), ('solar', 1070), ('observati...","[(('magnetic', 'field'), 473), (('solar', 'win...","[(('coronal', 'mass', 'ejections'), 48), (('ra...",Dirty
1,"Cruz, Kelle",Hunter College,"Cruz, Kelle",2004hst..prop10247C,['Resolving a Binary System that Straddles the...,2004-07-00,[],City University of New York Hunter College,We propose to make the first unequivocal resol...,['2004hst..prop10247C'],"[('dwarf', 6), ('system', 3), ('observation', ...","[(('propose', 'make'), 1), (('make', 'unequivo...","[(('propose', 'make', 'unequivocal'), 1), (('m...",Dirty
2,"Gawiser, Eric",Yale University,"Lin, C. -H., Bauer, Anne, Crowl, Hugh H., Cire...","2006A&A...460..597L, 2006AAS...209.2001B, 2006...",['Transition region counterpart of a moving ma...,"2006-12-00, 2006-12-00, 2006-12-00, 2006-12-00...","['Sun: granulation', 'Sun: magnetic fields', '...","Armagh Observatory, College Hill, Armagh BT61 ...",Context: .While moving magnetic features have ...,"['10.1051/0004-6361:20065688', '2006A&A...460....","[('galaxy', 683), ('star', 483), ('cluster', 3...","[(('black', 'hole'), 145), (('star', 'formatio...","[(('black', 'hole', 'mass'), 56), (('hubble', ...",Dirty


**Locate arXiv identifiers for papers**

For proof of functionality, we choose to focus on arXiv published papers, due to the easy to navigate API of arXiv. The identifier is pulled so that the pdf can be easily downloaded. Future work includes expanding on publication websites we pull from.

In [5]:
target = r'arXiv:(\S+)'

def process_row_id(row):
  identifiers = row['Identifier'].split(',')
  arXiv_list = []

  for identifier in identifiers:
    match = re.search(target, identifier)
    if match:
      arXiv = match.group(1)
      arXiv_list.append(arXiv)
    else:
      arXiv_list.append(None)
  if any(arXiv_list):
    return ', '.join(str(x) for x in arXiv_list if x is not None)
  else:
    return 'None'

datf['arxiv-id'] = datf.apply(process_row_id, axis = 1)
datf

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type,arxiv-id
Unnamed: 0,,,,,,,,,,,,,,,
0,"Browning, Matthew","University of California, Berkeley","Enoch, Melissa L., Hull, A. J., Oka, M., Salem...","2009ApJ...707..103E, 2009AGUFMSM41B1727H, 2009...",['Disk and Envelope Structure in Class 0 Proto...,"2009-12-00, 2009-12-00, 2009-12-00, 2009-12-00...","['infrared: ISM', 'ISM: individual: Serpens', ...","Department of Astronomy, University of Califor...",We present the first results of a program to c...,"['arXiv:0910.2715', '10.1088/0004-637X/707/1/1...","[('field', 1196), ('solar', 1070), ('observati...","[(('magnetic', 'field'), 473), (('solar', 'win...","[(('coronal', 'mass', 'ejections'), 48), (('ra...",Dirty,"0910.2715', 0909.0270', 0905.0001', 0911.0138'..."
1,"Cruz, Kelle",Hunter College,"Cruz, Kelle",2004hst..prop10247C,['Resolving a Binary System that Straddles the...,2004-07-00,[],City University of New York Hunter College,We propose to make the first unequivocal resol...,['2004hst..prop10247C'],"[('dwarf', 6), ('system', 3), ('observation', ...","[(('propose', 'make'), 1), (('make', 'unequivo...","[(('propose', 'make', 'unequivocal'), 1), (('m...",Dirty,None
2,"Gawiser, Eric",Yale University,"Lin, C. -H., Bauer, Anne, Crowl, Hugh H., Cire...","2006A&A...460..597L, 2006AAS...209.2001B, 2006...",['Transition region counterpart of a moving ma...,"2006-12-00, 2006-12-00, 2006-12-00, 2006-12-00...","['Sun: granulation', 'Sun: magnetic fields', '...","Armagh Observatory, College Hill, Armagh BT61 ...",Context: .While moving magnetic features have ...,"['10.1051/0004-6361:20065688', '2006A&A...460....","[('galaxy', 683), ('star', 483), ('cluster', 3...","[(('black', 'hole'), 145), (('star', 'formatio...","[(('black', 'hole', 'mass'), 56), (('hubble', ...",Dirty,"astro-ph/0607086', astro-ph/0604080', astro-ph..."


**Download PDF version of ArXiv papers**

Run time for 20 papers is approximately 10s.

In [6]:
download_directory = '/content/'
pdf_filenames = []

def download_pdf(identifier):
    pdf_url = f'https://arxiv.org/pdf/{identifier}.pdf'
    save_path = os.path.join(download_directory, f'{identifier.replace("/", "_")}.pdf')
    response = requests.get(pdf_url)

    if response.status_code == 200:
        with open(save_path, 'wb') as pdf_file:
            pdf_file.write(response.content)
        return save_path
    return None

all_identifiers = [identifier for row in datf['arxiv-id'].str.split(', ') for identifier in row if identifier != 'None']

with concurrent.futures.ThreadPoolExecutor(max_workers = 20) as executor:
    results = list(executor.map(download_pdf, all_identifiers))

pdf_filenames.extend(result for result in results if result is not None)

#print(pdf_filenames)

**Process Text Files**

Large Language Models need information passed through in the form of an ASCII file to perfom its functions properly. Information from the original pdf versions, like tables, references, acknowledgements, etc. can result in hallucinations in the model. Removing this information is vital after converting the infromation from pdf to txt.

In [7]:
def remove_text_in_brackets(text):
  pattern = r'\([^)]*\)|\[[^\]]*\]'
  return re.sub(pattern, '', text)

def remove_lines_starting_with_number_or_symbol(text):
  pattern = r'^[\d\W].*$'
  return re.sub(pattern, '', text, flags=re.MULTILINE)

def remove_lines_with_one_word(text):
  lines = text.split('\n')
  pattern = r'^\s*\w+\s*$'
  filtered_lines = [line for line in lines if not re.match(pattern, line)]
  return '\n'.join(filtered_lines)

def remove_empty_lines(text):
  lines = text.split('\n')
  non_empty_lines = [line for line in lines if line.strip() != '']
  return '\n'.join(non_empty_lines)

def find_and_remove_references(text, keywords):
  earliest_position = -1
  for keyword in keywords:
    position = text.find(keyword)
    if position != -1:
      earliest_position = position if earliest_position == -1 else min(position, earliest_position)
  if earliest_position != -1:
    text = text[:earliest_position]
  return text

def process_text(text):
  keywords = ["REFERENCES", "ACKNOWLEDGEMENTS", "References", "Acknowledgements"]

  text = find_and_remove_references(text, keywords)
  text = find_and_remove_references(text, keywords)
  text = remove_text_in_brackets(text)
  text = remove_lines_starting_with_number_or_symbol(text)
  text = remove_lines_with_one_word(text)
  text = remove_empty_lines(text)

  return text

def extract_and_process_text(pdf_path, output_path):
  pdf_doc = fitz.open(pdf_path)
  text = ''
  for page in  pdf_doc:
    text += page.get_text()

  lines = text.split('\n')
  filtered_lines = [line for line in lines if len(line.strip()) > 1]
  filtered_text = '\n'.join(filtered_lines)
  cleaned_text = process_text(filtered_text)

  with open(output_path, 'w', encoding = 'utf-8') as output_file:
    output_file.write(cleaned_text)
  return output_path

def process_pdf(pdf_filename):
  text_filename = pdf_filename.replace('.pdf','.txt')
  cleaned_text = extract_and_process_text(pdf_filename, text_filename)
  return text_filename

text_filenames = []

num_workers = 100

with concurrent.futures.ThreadPoolExecutor(max_workers = num_workers) as executor:
  results = list(executor.map(process_pdf, pdf_filenames))

text_filenames.extend(result for result in results if result is not None)

#print(text_filenames)

**Summarization Model**

In [10]:
hf_tag = "pszemraj/led-large-book-summary"

_device = torch.cuda.current_device() if torch.cuda.is_available() else torch.device("cpu")

model = AutoModelForSeq2SeqLM.from_pretrained(hf_tag).to(_device)

tokenizer = AutoTokenizer.from_pretrained(hf_tag)

token_batch_length = 3072
batch_stride = 50
max_len_ratio = 8

settings = {
    'min_length': 5,
    'max_length': int(token_batch_length // max_len_ratio),
    'no_repeat_ngram_size': 7,
    'encoder_no_repeat_ngram_size' : 7,
    'repetition_penalty': 3.7,
    'num_beams': 12,
    'length_penalty': 0.5,
    'early_stopping': True,
    'do_sample': False}

logging.info(f"using textgen params:\n\n:{settings}")

def summarize(ids, mask, **kwargs):
    ids = ids[None, :]
    mask = mask[None, :]

    input_ids = ids.to(_device)
    attention_mask = mask.to(_device)
    global_attention_mask = torch.zeros_like(attention_mask)
    global_attention_mask[:, 0] = 1

    summary_pred_ids = model.generate(
        ids,
        attention_mask=mask,
        global_attention_mask=global_attention_mask,
        return_dict_in_generate=True,
        **kwargs)
    summary = tokenizer.batch_decode(
        summary_pred_ids.sequences,
        skip_special_tokens=True,
        remove_invalid_values=True,)

    return summary

def summarize_via_tokenbatches(input_text, **kwargs):
    encoded_input = tokenizer(
        input_text,
        padding='max_length',
        truncation=True,
        max_length=token_batch_length,
        stride=batch_stride,
        return_overflowing_tokens=True,
        add_special_tokens=False,
        return_tensors='pt',)

    in_id_arr, att_arr = encoded_input.input_ids.to(_device), encoded_input.attention_mask.to(_device)
    gen_summaries = []

    with tqdm(total=len(in_id_arr)) as pbar:
        for _id, _mask in zip(in_id_arr, att_arr):
            result = summarize(ids=_id, mask=_mask, **kwargs)
            _sum = {"input_tokens": _id, "summary": result}
            gen_summaries.append(_sum)
            pbar.update()

    return gen_summaries

directory = ''
text_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.txt')]


for file_path in text_files:
  with open(file_path, 'r', errors='ignore') as f:
    raw_text = f.read()

  long_text = clean(raw_text)
  logging.info(f"removed {len(long_text) - len(raw_text)} chars via cleaning")

  encoded_input = tokenizer(
    long_text,
    padding='max_length',
    truncation=True,
    max_length=token_batch_length,
    stride=batch_stride,
    return_overflowing_tokens=True,
    add_special_tokens =False,
    return_tensors='pt',)

  _summaries = summarize_via_tokenbatches(long_text, **settings)

  sum_text = [s["summary"][0] for s in _summaries]
  full_summary = "\n\t".join(sum_text)

  path = Path(file_path)
  _outpath = f"SUM_{path.name}"

  with open(_outpath,"w",) as fo:
    fo.writelines(full_summary)
    fo.write("\n" * 3)
    fo.write("\n\n---\n")

  if torch.cuda.is_available():
    torch.cuda.empty_cache()

num_workers = 4

with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    executor.map(summarize, text_files)

!cat $_outpath

c:\Users\izzye\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


FileNotFoundError: [WinError 3] The system cannot find the path specified: ''

**Combine Summaries for Given Author**

To understand the expertise of the author, we need to include all possible publications, so that we see an overarching view of their work.

In [ ]:
datf['Summaries'] = ""

for index,row in datf.iterrows():
  identifiers = row['arxiv-id'].split(',')

  concat_summ = ""

  for identifier in identifiers:
    if identifier.strip() == 'None':
      concat_summ = 'None'
      break

    sanitized_identifier = identifier.strip().replace('/', '_')

    filename = f'SUM_{sanitized_identifier}.txt'

    if os.path.exists(filename):
      with open(filename, 'r', encoding = 'utf-8') as file:
        concat_summ += file.read()
    else:
      concat_summ = 'File not found'

  datf.at[index, 'Summaries'] = concat_summ

In [ ]:
datf.reset_index(inplace = True)

**N_grams**

In [ ]:
def sum_n_grams(df, directorypath): #directory path should lead to TextAnalysis.py
    top10Dict = {'Summaries Top 10 Words':[],
                 'Summaries Top 10 Bigrams':[],
                 'Summaries Top 10 Trigrams':[]}

    for i in df.values:
        summaries = i[15]

        top10words = TA.topwords(summaries, directorypath)
        top10bigrams = TA.topbigrams(summaries, directorypath)
        top10trigrams = TA.toptrigrams(summaries, directorypath)

        top10Dict['Summaries Top 10 Words'].append(top10words)
        top10Dict['Summaries Top 10 Bigrams'].append(top10bigrams)
        top10Dict['Summaries Top 10 Trigrams'].append(top10trigrams)

    sum_top10Df = df
    sum_top10Df['Summaries Top 10 Words'] = top10Dict['Summaries Top 10 Words']
    sum_top10Df['Summaries Top 10 Bigrams'] = top10Dict['Summaries Top 10 Bigrams']
    sum_top10Df['Summaries Top 10 Trigrams'] = top10Dict['Summaries Top 10 Trigrams']

    sum_top10Df = sum_top10Df[['Input Author', 'Input Institution', 'First Author', 'Bibcode', 'Title', 'Publication Date',
             'Keywords', 'Affiliations', 'Abstract', 'Top 10 Words', 'Top 10 Bigrams', 'Top 10 Trigrams', 'Data Type',
            'arxiv-id', 'Summaries', 'Summaries Top 10 Words', 'Summaries Top 10 Bigrams', 'Summaries Top 10 Trigrams']]

    return sum_top10Df

In [ ]:
path_stop= '/content/'
stop_file='stopwords.txt'
stop_dir=path_stop+stop_file

final_df = sum_n_grams(datf, stop_dir)
#final_df

**Save New File**

Change the name from results to preferred naming convention.

In [ ]:
final_df.to_csv('Results.csv', index = False)